In [ ]:
# パッケージ更新の場合にはこれを実施
# !rye sync

In [ ]:
import json
import pathlib
from pprint import pprint
import sys

import openai
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from tqdm.auto import tqdm
import gspread

from judge_gpt import JudgeGPT, ColumnInfo
from judge_gpt.auth import auth_gcloud, set_openai_apikey_from_file
from judge_gpt.conf import read_conf
from judge_gpt.gss_reader import GSSReader


def plot_judge_stats(df_judge: pd.DataFrame, players: list[str]):
    agg_df = df_judge.groupby("judge", as_index=False).agg(
        player_count=("player_name", lambda x : len(set(x))),
        stats_difficulty_mean=("difficulty", "mean"),
        stats_variation_mean=("variation", "mean"),
        stats_refined_mean=("refined", "mean"),
        stats_performance_mean=("performance", "mean"),
        stats_difficulty_std=("difficulty", "std"),
        stats_variation_std=("variation", "std"),
        stats_refined_std=("refined", "std"),
        stats_performance_std=("performance", "std"),
    )
    
    assert (agg_df["player_count"] == len(players)).all(), "Judge result is missing"
    
    agg_df = agg_df.set_index("judge").T
    display(agg_df)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
    agg_df.loc[[
        "stats_difficulty_mean",
        "stats_variation_mean",
        "stats_refined_mean",
        "stats_performance_mean",
    ], :].plot.bar(ax=ax1)
    
    agg_df.loc[[
        "stats_difficulty_std",
        "stats_variation_std",
        "stats_refined_std",
        "stats_performance_std",
    ], :].plot.bar(ax=ax2)
    ax1.grid()
    ax2.grid()

## 初期設定
認証情報やらを読み込む

In [ ]:
# 設定ファイル
conf_judge, conf_response_schema = read_conf(
    "./conf/demo/judge.yaml",
    "./conf/demo/response_schema.yaml",
)

# 認証
gss_reader = GSSReader(conf_judge["gss"])
set_openai_apikey_from_file(conf_judge["chatgpt"]["secrets"])

### 書き出し先のSpreadSheetを作成
スプレッドシートを作成しておく。一度作成したら作成する必要はない。

In [ ]:
# 審査シートを作成
# folder_idはスプレッドシートを書き出し先のGoogle driverのFolderのID
# gss_reader.gc.create("judge_result", folder_id="1oQVk8aWfiXb0sJH8lLEq_fsZNMXcYRby")

## 読み込みと前処理

### 読み込むデータについて
Spread Sheetはセクション機能を用いて以下のような構造になっている
- 審査員名を入力する
- 選手Aに対する審査
- 選手Bに対する審査
- ...
- 選手xに対する審査

Spread Sheetには以下のような形式で出力される。

| タイムスタンプ | メールアドレス | 審査部門 | 回答者 | 難易度 | 多彩性度 | 操作安定度 | 演技構成度 | ... | 難易度 | 多彩性度 | 操作安定度 | 演技構成度 |
| --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- | 
| yyyy/mm/dd hh:mm:ss | hoge@email.address | トス | ジャッジ1 | hogehoge | fugafuga | piyopiyo | ... | ... | ... | ... | ... | ... | 

セクション分けしても、セクションの情報はカラムには乗らないので、以下のような縦持ちに変換する必要がある。

| タイムスタンプ | メールアドレス | 審査部門 | 回答者 | 選手名 | 難易度 | 多彩性度 | 操作安定度 | 演技構成度 |
| --- |  --- |  --- |  --- |  --- |  --- |  --- |  --- | --- |
| yyyy/mm/dd hh:mm:ss | hoge@email.address | トス | ジャッジ1 | 選手A |hogehoge | fugafuga | piyopiyo | ... |

In [ ]:
judge_result, gss_conf = gss_reader.get_worksheet("toss")

display(judge_result)

In [ ]:
# テスト用のレコードを省く
judge_result = judge_result.query("回答者 != '運営'")

# 選手の数とジャッジの数
print("部門")
judge_list = judge_result["回答者"].unique()
print("審査員: ", ",".join(judge_list))
print(f"選手数: {len(gss_conf.player_order):d}")
display(judge_result.groupby("回答者", as_index=False)["選手"].nunique())

## 審査

In [ ]:
gpt = JudgeGPT(
    model=conf_judge["chatgpt"]["model"],
    conf_response_schema=conf_response_schema,
    seed=conf_judge["chatgpt"]["seed"],
    column_info=ColumnInfo(judge="回答者", player="選手"),
)

In [ ]:
# promptの作成
prompts = gpt.make_prompt(
    judge_list, 
    judge_result,
    gss_conf=gss_conf,
    system_prompt=conf_judge["chatgpt"]["prompt_judge"].format(template=gss_conf.template)
)

In [ ]:
# chatgptへリクエスト
res_log, df_judge_result_all = gpt.call_chatgpt(prompts)

In [ ]:
# テーブルの表示
display(df_judge_result_all)

# 審査情報の可視化
plot_judge_stats(df_judge_result_all, gss_conf.player_order)

# 選手毎の平均点を計算
df_judge_result_all.groupby("player_name", as_index=False)[["difficulty", "variation", "refined", "performance"]].mean()

### 審査結果書き出し¶

In [ ]:
judge_result_sheet = gss_reader.gc.open_by_key(conf_judge["gss"]["output"]["key"])

In [ ]:
try: 
    ws = judge_result_sheet.worksheet("トスジャグリング")
except gspread.WorksheetNotFound as e:
    ws = judge_result_sheet.add_worksheet(title="トスジャグリング", rows=200, cols=200)
except Exception as e:
    raise e

In [ ]:
gss_reader.insert_dataframe_to_ws(
    ws,
    df_judge_result_all,
    add_filter=True,
)

### 審査結果要約
UIでChatGPTに投げる用の情報に変換

一番最初のプロンプト
```
あなたはジャグリングの大会で、審査員の役割を担うChatGPTです。大会では、パフォーマンスごとに審査員が採点項目に基づいて評価コメントを記述します。あなたはこれらのコメントと採点項目をもとに、各選手のパフォーマンスを点数化し、その評価を要約して説明する役割を持ちます。選手ごとに複数の審査員から提供される評価コメントを結合して渡すので、審査員の総評として要約し説明することが求められます。
元となる評価コメントは以下の観点で作成されています。
1. 難易度
2. 多彩性度
3. 操作安定度
4. 演技構成
です。
```

In [ ]:
players = df_judge_result_all["player_name"].unique()
for player in players:
    print(f"{player}: -----------------")
    descriptions = df_judge_result_all.query(f"player_name == '{player}'")["desc"].unique()
    print("\n".join(descriptions))
    